# MLflow

Este es el reporte para MLFlow para la práctica de despliegue, en este notebook pegaré y guardaré las información de esta práctica

In [47]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.datasets import load_diabetes

In [2]:
diabetes = load_diabetes()

## Análisis de Datos

Afortunadamente skearn.dataset tiene datasets muy chulos, no necesita gran analis. Más bien buscaré varios algoritmos para comparar.

In [3]:
df = pd.DataFrame(diabetes['data'], columns=diabetes['feature_names'])

In [4]:
df['target'] = diabetes['target']
df.head()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
0,0.038076,0.050680,0.061696,0.021872,-0.044223,-0.034821,-0.043401,-0.002592,0.019907,-0.017646,151.0
1,-0.001882,-0.044642,-0.051474,-0.026328,-0.008449,-0.019163,0.074412,-0.039493,-0.068332,-0.092204,75.0
2,0.085299,0.050680,0.044451,-0.005670,-0.045599,-0.034194,-0.032356,-0.002592,0.002861,-0.025930,141.0
3,-0.089063,-0.044642,-0.011595,-0.036656,0.012191,0.024991,-0.036038,0.034309,0.022688,-0.009362,206.0
4,0.005383,-0.044642,-0.036385,0.021872,0.003935,0.015596,0.008142,-0.002592,-0.031988,-0.046641,135.0


In [5]:
df.shape

(442, 11)

In [6]:
df.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-2.511817e-19,1.230790e-17,-2.245564e-16,-4.797570e-17,-1.381499e-17,3.918434e-17,-5.777179e-18,-9.042540e-18,9.293722e-17,1.130318e-17,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123988e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260971e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665608e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324559e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670422e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947171e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564379e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243232e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320436e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335973e-01,1.356118e-01,346.000000


In [7]:
df.columns

Index(['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6',
       'target'],
      dtype='object')

Siguendo el ejemplo guardo los ficheros entre test y train

In [8]:
X_train, X_test = train_test_split(df, test_size=0.25)

y_test = X_test['target']
X_test[['target']].to_csv('test-target.csv', index=False)
del X_test['target']
X_test.to_csv('test.csv', index=False)

y_train = X_train['target']
X_train[['target']].to_csv('train-target.csv', index=False)
del X_train['target']
X_train.to_csv('train.csv', index=False)

In [9]:
X_train

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6
305,-0.009147,0.050680,-0.039618,-0.040099,-0.008449,0.016222,-0.065491,0.071210,0.017765,-0.067351
80,0.070769,-0.044642,0.012117,0.042529,0.071357,0.053487,0.052322,-0.002592,0.025395,-0.005220
357,-0.060003,0.050680,0.049840,0.018430,-0.016704,-0.030124,-0.017629,-0.002592,0.049770,-0.059067
355,-0.074533,0.050680,-0.009439,0.014987,-0.037344,-0.021669,-0.013948,-0.002592,-0.033246,0.011349
259,0.012648,-0.044642,-0.036385,0.042529,-0.013953,0.012934,-0.026833,0.005157,-0.043984,0.007207
...,...,...,...,...,...,...,...,...,...,...
384,0.034443,0.050680,-0.029918,0.004658,0.093372,0.086994,0.033914,-0.002592,0.024055,-0.038357
126,-0.089063,-0.044642,-0.061174,-0.026328,-0.055231,-0.054549,0.041277,-0.076395,-0.093937,-0.054925
20,-0.049105,-0.044642,-0.056863,-0.043542,-0.045599,-0.043276,0.000779,-0.039493,-0.011897,0.015491
413,-0.052738,-0.044642,-0.000817,-0.026328,0.010815,0.007141,0.048640,-0.039493,-0.035816,0.019633


## Entrenamiento 1

In [45]:
model_name = "diabetes-predictor-IA3"

In [46]:
def rfrRun(n_estimator,min_samples_leaf,max_depth,max_feature, saveModel = False):
    """
    Función que genera un modelo RandomForest, entrena y registra en mlFlow el resultado y el modelo
    """

    # Modelo:
    clf = RandomForestRegressor(n_estimators=n_estimator,
                                min_samples_leaf=min_samples_leaf,
                                max_depth=max_depth,
                                max_features=max_feature,
                                random_state=1234)
    preprocessor = Pipeline(steps=[('scaler', StandardScaler())])
    model = Pipeline(steps=[('preprocessor', preprocessor),
                            ('RandomForestRegressor', clf)])
    
    # Emtrenamiento
    model.fit(X_train, y_train)

    # Evaluación
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # ML Flow FTW
    mlflow.set_experiment("Diabetes_IA3")
    with mlflow.start_run(run_name="Diabetes-RFR-IA3"):
        mlflow.log_params({
                "n_estimators": n_estimator,
                "min_samples_leaf": min_samples_leaf,
                "max_depth": max_depth,
                "max_features": max_feature
            })
        mlflow.log_metrics({ 
                # Error cuadratico medio
                'MSE_train': mean_squared_error(y_train, y_train_pred),
                'MSE_test': mean_squared_error(y_test, y_test_pred),
                
                # R2:
                'R2_train': r2_score(y_train, y_train_pred),
                'R2_test': r2_score(y_test, y_test_pred),
                
                # Raíz del error cuadrático medio
                'RMSE_train': np.sqrt(mean_squared_error(y_train, y_train_pred)),
                'RMSE_test': np.sqrt(mean_squared_error(y_test, y_test_pred)),
            })
        if saveModel:
            mlflow.sklearn.log_model(model, "RFRegressor", registered_model_name=model_name)
        else:
            mlflow.sklearn.log_model(model, "RFRegressor")


In [42]:
n_estimators = [100, 500, 1000]
max_depths = [5, 10]
min_samples_leafs = [2, 4]
max_features = ["sqrt","log2"]

for n_estimator in n_estimators:
    for max_depth in max_depths:
        for min_samples_leaf in min_samples_leafs:
            for max_feature in max_features:
                rfrRun(n_estimator,min_samples_leaf,max_depth,max_feature)

2025/06/13 02:01:29 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/13 02:01:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/13 02:01:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/13 02:01:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/13 02:01:37 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/13 02:01:39 

### Analisis de ejecución

Tras la ejecución de los modelos con distintas parámetricas para RandomForest revisamos su registro:

![image](./images//RandomForestList.png)

Se observa que la mejor paramétrica es:

![image](./images/descarga.png)
![image](./images/RFparameters.png)

Pero como se puede ver no es muy bueno, igualmente se guarda el modelo

### Registro y Despliegue de Modelo con MLflow


In [43]:
rfrRun(500,2,10,"sqrt", saveModel = True)

2025/06/13 02:02:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'diabetes-predictor-IA3' already exists. Creating a new version of this model...
Created version '3' of model 'diabetes-predictor-IA3'.


### Modelo Guardado

![image](./images/guardadoRFR.png)

Lo ejecuté 2 veces sin querer, y he pasado a producción el segundo

Ahora probamos que se guardara bien.

In [44]:
# Cargar el modelo de producción
model = mlflow.pyfunc.load_model(f"models:/diabetes-predictor-IA3/Production")

model.predict(X_test)


array([ 90.89700301, 135.2563596 , 153.92975958, 171.7622302 ,
       174.04363389, 147.05917665, 190.38576012, 134.29991195,
       143.91376146, 141.47207834, 215.76273278, 153.36919711,
       104.63555652, 157.98331984, 103.06398792, 108.85519862,
       132.903362  , 141.95204645, 196.45016036,  78.40645614,
       115.98947039, 102.42166316, 124.6820402 , 189.53818297,
       101.86294364, 102.22770854, 170.582668  , 177.75020223,
       217.16053058, 110.53828316, 150.23131655, 135.08113727,
       111.10689979, 165.23524471, 239.19449698, 174.22304522,
       201.63559101, 103.79219522, 135.62349834, 171.50664987,
       219.86185   , 127.35868915, 208.28956057,  87.3041782 ,
       161.44000896,  81.33646893, 167.30426611, 219.00088304,
       114.67709176, 122.17996007,  99.15953598, 135.69793304,
       219.161583  , 156.55389534, 223.99959583, 228.36758224,
       212.32709439, 130.03406214, 238.11135159,  92.03246842,
       102.71746884, 250.48766899, 118.80143959, 171.88

## Entrenamiento 2

In [48]:
def xgbRun(learning_rate, max_depth, n_estimators, subsample, colsample_bytree, saveModel=False):
    """
    Función que genera un modelo XGBoost, entrena y registra en mlFlow el resultado y el modelo
    """
    # Modelo
    xgb_model = xgb.XGBRegressor(
        learning_rate=learning_rate,
        max_depth=max_depth,
        n_estimators=n_estimators,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        random_state=1234
    )
    
    preprocessor = Pipeline(steps=[('scaler', StandardScaler())])
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('XGBRegressor', xgb_model)
    ])
    
    # Entrenamiento
    model.fit(X_train, y_train)

    # Evaluación
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # ML Flow FTW
    mlflow.set_experiment("Diabetes_IA3")
    with mlflow.start_run(run_name="Diabetes-XGB-IA3"):
        mlflow.log_params({
            "learning_rate": learning_rate,
            "max_depth": max_depth,
            "n_estimators": n_estimators,
            "subsample": subsample,
            "colsample_bytree": colsample_bytree
        })
        
        mlflow.log_metrics({ 
            # Error cuadrático medio
            'MSE_train': mean_squared_error(y_train, y_train_pred),
            'MSE_test': mean_squared_error(y_test, y_test_pred),
            
            # R²
            'R2_train': r2_score(y_train, y_train_pred),
            'R2_test': r2_score(y_test, y_test_pred),
            
            # Raíz del error cuadrático medio
            'RMSE_train': np.sqrt(mean_squared_error(y_train, y_train_pred)),
            'RMSE_test': np.sqrt(mean_squared_error(y_test, y_test_pred)),
        })
        
        if saveModel:
            mlflow.sklearn.log_model(model, "XGBRegressor", registered_model_name="diabetes-predictor-xgb-IA3")
        else:
            mlflow.sklearn.log_model(model, "XGBRegressor")



In [49]:
# Parámetros para probar
learning_rates = [0.01, 0.1]
max_depths = [3, 5, 7]
n_estimators_list = [100, 500]
subsamples = [0.8, 1.0]
colsample_bytrees = [0.8, 1.0]

# Bucle de pruebas
for lr in learning_rates:
    for md in max_depths:
        for ne in n_estimators_list:
            for ss in subsamples:
                for cb in colsample_bytrees:
                    print(f"\nProbando con:")
                    print(f"learning_rate: {lr}")
                    print(f"max_depth: {md}")
                    print(f"n_estimators: {ne}")
                    print(f"subsample: {ss}")
                    print(f"colsample_bytree: {cb}")
                    
                    xgbRun(lr, md, ne, ss, cb)



Probando con:
learning_rate: 0.01
max_depth: 3
n_estimators: 100
subsample: 0.8
colsample_bytree: 0.8


2025/06/13 02:05:35 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Probando con:
learning_rate: 0.01
max_depth: 3
n_estimators: 100
subsample: 0.8
colsample_bytree: 1.0


2025/06/13 02:05:36 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Probando con:
learning_rate: 0.01
max_depth: 3
n_estimators: 100
subsample: 1.0
colsample_bytree: 0.8


...

2025/06/13 02:07:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Probando con:
learning_rate: 0.1
max_depth: 7
n_estimators: 500
subsample: 1.0
colsample_bytree: 0.8


2025/06/13 02:07:06 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



Probando con:
learning_rate: 0.1
max_depth: 7
n_estimators: 500
subsample: 1.0
colsample_bytree: 1.0


2025/06/13 02:07:08 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


### Ahora vemos el registro de XGB 

![image](./images/XGBList.png)

Reviso el mejor, no tan bueno como esperaba

![image](./images/XGB_best.png)

In [50]:

# Guardar el mejor modelo
xgbRun(0.01, 3, 500, 0.8, 0.8, saveModel=True)

2025/06/13 02:14:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Registered model 'diabetes-predictor-xgb-IA3' already exists. Creating a new version of this model...
Created version '1' of model 'diabetes-predictor-xgb-IA3'.


### Revisamos y probamos

![image](./images/XGBModelo.png)

In [51]:
# Cargar el modelo de producción
model = mlflow.pyfunc.load_model(f"models:/diabetes-predictor-xgb-IA3/Production")

model.predict(X_test)

array([ 81.68191 , 153.10175 , 145.70717 , 179.76581 , 169.46591 ,
       153.81657 , 194.17865 , 131.07669 , 126.514786, 147.14337 ,
       212.453   , 165.44662 ,  95.56344 , 176.10126 ,  84.02394 ,
       110.32637 , 130.31895 , 136.70865 , 204.24959 ,  76.10729 ,
       110.69276 , 108.823975, 121.23394 , 213.67955 ,  99.87983 ,
        98.81055 , 161.90404 , 183.59686 , 226.64207 , 115.33466 ,
       158.81593 , 129.952   , 107.13852 , 164.60857 , 257.63947 ,
       174.47183 , 203.14844 , 102.35734 , 144.31227 , 183.51051 ,
       222.95973 , 129.20844 , 217.67336 ,  91.920944, 166.93617 ,
        72.86546 , 167.83296 , 231.15385 , 110.01879 , 124.118744,
       105.99609 , 137.05058 , 225.93893 , 154.99684 , 225.01228 ,
       222.97531 , 222.34438 , 127.79173 , 267.27652 ,  84.45804 ,
        92.8449  , 277.31607 , 124.52757 , 161.8739  , 152.35814 ,
       139.24065 , 197.51326 , 249.93526 ,  85.01098 , 135.07584 ,
       102.91612 , 175.40257 , 257.05032 , 105.118576,  87.029